In [ ]:
import requests

client_id = "gnnWto0tTISYhC1m9Ay8"
client_secret = "bPKp6UWmLTPXUzXMXYt-kwjs2Bg_4j19A_D9G_vt"

auth_resp = requests.post(
    "https://openapi.vito.ai/v1/authenticate",
    data={
        "client_id": client_id,
        "client_secret": client_secret
    }
)
jwt = auth_resp.json()['access_token']

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np
import requests
import json
import time

# ====== 설정 ======
SAMPLE_RATE = 16000            # RTZR이 잘 받는 표준 샘플레이트
DURATION = 5                   # 녹음 시간(초)
OUTPUT_FILE = "record.wav"     # 저장 파일명
JWT = jwt
# ==================


def record_audio():
    print("🎙️ 녹음 시작... (말하세요)")
    audio = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='int16')
    sd.wait()
    write(OUTPUT_FILE, SAMPLE_RATE, audio)
    print("🎙️ 녹음 완료 → record.wav 저장됨")


def transcribe_audio():
    config = {
        "model_name": "sommers",
        "language": "ko",
        "use_paragraph_splitter": True,
    }

    resp = requests.post(
        "https://openapi.vito.ai/v1/transcribe",
        headers={"Authorization": f"Bearer {JWT}"},
        data={"config": json.dumps(config)},
        files={"file": open(OUTPUT_FILE, "rb")},
    )

    return resp.json().get("id")


def poll_transcription(transcribe_id):
    """RTZR은 async라서 polling 필요"""
    print("⏳ 결과 대기 중...")

    while True:
        resp = requests.get(
            f"https://openapi.vito.ai/v1/transcribe/{transcribe_id}",
            headers={"Authorization": f"Bearer {JWT}"},
        )
        data = resp.json()

        status = data.get("status")
        print("현재 상태:", status)

        if status == "completed":
            print("\n===== 🎉 인식 결과 =====")
            for utt in data["results"]["utterances"]:
                print(f"[spk {utt.get('spk', '?')}] {utt['msg']}")
            print("========================\n")
            return

        if status == "failed":
            print("❌ 전사 실패:", data)
            return

        time.sleep(0.3)



# record_audio()
tid = transcribe_audio()
poll_transcription(tid)
